# LangChain Agents Bootcamp - Financial Analysis with Gemini

Complete guide to building production-ready agents with memory, middleware, and streaming.

## Overview

This notebook covers:
- **Short-term Memory**: Persist conversation state with SQLite
- **Built-in Middleware**: Production patterns (summarization, limits, PII detection, todo tracking)
- **Structured Output**: Type-safe agent responses with Pydantic
- **Streaming Modes**: Real-time updates (`messages`, `updates`, `values`)

## Setup

Initialize model and tools for financial analysis.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from scripts import base_tools

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

system_prompt = """You are a financial analyst specializing in tech stocks.
Provide data-driven analysis with clear insights."""

## 1. Basic Agent

Create a simple agent with tools but no memory.

In [ ]:
agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt
)

response = agent.invoke({
    "messages": [HumanMessage("What's Apple's current stock price?")]
})

print(response["messages"][-1].text)

In [ ]:
response

In [ ]:
response["messages"][-1].content_blocks

In [ ]:
response = agent.invoke({
    "messages": ["What's Apple's current stock price?"]
})
response

## 2. Short-term Memory with SQLite

Add conversation persistence using SQLite checkpointer. Agent remembers previous turns within a session.

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

conn = sqlite3.connect("data/financial_agent.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent_memory = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

In [ ]:
config = {"configurable": {"thread_id": "session_1"}}

# First turn
response = agent_memory.invoke({
    "messages": [HumanMessage("Search for Apple's latest earnings")]
}, config)
print(response["messages"][-1].text)

In [ ]:
from langchain.agents.middleware import SummarizationMiddleware

agent_summary = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(
            model="gemini-2.5-flash",
            trigger=[("tokens", 4000), ("messages", 10)],
            keep=("messages", 4)
        )
    ]
)

In [ ]:
response

## Built-in Middleware

## 3. Middleware: Summarization

Automatically summarize old messages when history grows too long.

**Use Case**: Long conversations that exceed context windows.

In [ ]:
from langchain.agents.middleware import SummarizationMiddleware

agent_summary = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(
            model=model,
            trigger= ("messages", 10),
            keep=("messages", 5)
        )
    ]
)

In [ ]:
queries = [
    "Search for Apple stock",
    "What about Microsoft?",
    "Compare their P/E ratios"
]

for query in queries:
    response = agent_summary.invoke({"messages": [HumanMessage(query)]}, config)
    print(f"Q: {query}\nA: {response['messages'][-1].content}\n")

state = agent_summary.get_state(config)
print(f"Total messages: {len(state.values['messages'])}")

In [ ]:
response

## 4. Middleware: Model and Tool Call Limit

Limit the number of model calls per request to prevent runaway costs.

**Exit Behaviors**: `"end"` (stop) or `"continue"` (proceed without model)

In [ ]:
from langchain.agents.middleware import ModelCallLimitMiddleware
from langchain.agents.middleware import ToolCallLimitMiddleware
from langchain.agents.middleware import ModelFallbackMiddleware

agent_limit = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    middleware=[
        ModelCallLimitMiddleware(
            run_limit=2,
            exit_behavior="end"
        ),
        ToolCallLimitMiddleware(
            run_limit=2,
            exit_behavior="continue"
            
        ),
                ModelFallbackMiddleware(ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite"))
    ]
)

response = agent_limit.invoke({
    "messages": [HumanMessage("what is the latest apple stock price and what is the latest weather in mumbai?")]
})
print("Limited calls:", response["messages"][-1].text)

## 5. Middleware: Guardrails and PII Detection

Automatically detect and redact/mask personally identifiable information.

**Strategies**: `"redact"` (remove), `"mask"` (replace with ***), `"block"` (prevent request)

In [ ]:
from langchain.agents.middleware import PIIMiddleware

agent_pii = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        PIIMiddleware("api_key", detector=r"sk-[a-zA-Z0-9]{32}", strategy="block", apply_to_input=True)
    ]
)

response = agent_pii.invoke({
    "messages": [HumanMessage("Hi my name is laxmi kant tiwari and my email is test@example.com for Apple updates")]
})
print(response["messages"][-1].text)

## 6. Middleware: Todo List

Track and manage multi-step tasks within the agent.

In [ ]:
from langchain.agents.middleware import TodoListMiddleware

agent_todo = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[TodoListMiddleware()]
)

config = {"configurable": {"thread_id": "todo_session"}}

response = agent_todo.invoke({
    "messages": [HumanMessage("Analyze Apple revenue and compare competitors")]
}, config)
print(response["messages"][-1].text)

## 7. Structured Output

Return type-safe Pydantic models from agent responses.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class FinancialAnalysis(BaseModel):
    """Structured financial analysis."""
    company: str = Field(description="Company name")
    stock_symbol: str = Field(description="Stock ticker")
    current_price: Optional[str] = Field(description="Current price")
    analysis: str = Field(description="Brief analysis")
    recommendation: str = Field(description="Buy/Hold/Sell")

In [ ]:
agent_structured = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt="Provide structured financial analysis.",
    checkpointer=checkpointer,
    response_format=FinancialAnalysis
)

In [ ]:
config = {"configurable": {"thread_id": "structured_session"}}

response = agent_structured.invoke({
    "messages": [HumanMessage("Analyze Apple stock")]
}, config)

# Find the ToolMessage with structured output
for msg in reversed(response["messages"]):
    if msg.name == "FinancialAnalysis":
        # Parse the content to extract structured data
        content = msg.content
        
        # Display in a clean format
        print("=" * 50)
        print("STRUCTURED FINANCIAL ANALYSIS")
        print("=" * 50)
        
        # Extract key-value pairs from the ToolMessage content
        import re
        matches = re.findall(r"(\w+)='([^']*)'", content)
        for key, value in matches:
            print(f"{key.replace('_', ' ').title()}: {value}")
        
        print("=" * 50)
        break
else:
    # Fallback to regular content
    print("Response:", response["messages"][-1].content)

In [ ]:
last_message

## 8. Streaming: Messages Mode
Three streaming modes for real-time agent updates:
- **`messages`**: Stream individual messages as they're generated
- **`updates`**: Stream state updates after each step
- **`values`**: Stream complete state values

In [ ]:
agent_stream = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "stream_1"}}

print("\n=== Stream: Messages ===")
for chunk in agent_stream.stream({
    "messages": [HumanMessage("Quick Apple stock update")]
}, config):
    print(chunk)
    print("---")

In [ ]:
config = {"configurable": {"thread_id": "stream_3"}}

print("\n=== Stream: Values ===")
for chunk in agent_stream.stream({
    "messages": [HumanMessage("Compare Apple and Microsoft")]
}, config, stream_mode="values"):
    if "messages" in chunk:
        print(f"State: {len(chunk['messages'])} messages")
    print("---")